# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [38]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [39]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [40]:
def build_classifiers():
    lr = LinearRegression()
    knn = KNeighborsClassifier()
    svm = SVC()
    dt = DecisionTreeClassifier()
    nb = GaussianNB()
    qda = QuadraticDiscriminantAnalysis()
    
    classifiers = [lr, knn, svm, dt, nb, qda]
    names = ["LinearRegression", "KNeighbors", "SVM", "DecisionTree", "NaiveBayes", "QDA"]
    
    for clf in classifiers:
        clf.fit(data_set, labels)
    
    accuracies = []
    for i, clf in enumerate(classifiers):
        if names[i] == "LinearRegression":
            pred = np.round(clf.predict(test_data_set)).astype(int)
        else:
            pred = clf.predict(test_data_set)
        
        acc = accuracy_score(test_labels, pred)
        accuracies.append(acc)
        print(f"{names[i]} accuracy: {acc:.4f}")
    
    top3_indices = np.argsort(accuracies)[-3:]
    best_classifiers = [classifiers[i] for i in top3_indices]
    best_names = [names[i] for i in top3_indices]
    
    print(f"\nSelected top 3 classifiers: {best_names}")
    
    return best_classifiers[0], best_classifiers[1], best_classifiers[2]

In [41]:
def build_stacked_classifier(classifiers):
    output = []
    for i, classifier in enumerate(classifiers):
        if isinstance(classifier, LinearRegression):
            pred = np.round(classifier.predict(data_set)).astype(int)
            output.append(pred)
        else:
            output.append(classifier.predict(data_set))
    
    output = np.array(output).reshape((130, 3))
    
    stacked_classifier = DecisionTreeClassifier()  
    stacked_classifier.fit(output.reshape((130, 3)), labels.reshape((130,)))
    
    test_set = []
    for i, classifier in enumerate(classifiers):
        if isinstance(classifier, LinearRegression):
            pred = np.round(classifier.predict(test_data_set)).astype(int)
            test_set.append(pred)
        else:
            test_set.append(classifier.predict(test_data_set))
            
    test_set = np.array(test_set).reshape((len(test_set[0]), 3))
    predicted = stacked_classifier.predict(test_set)
    
    return predicted

In [42]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

LinearRegression accuracy: 0.9000
KNeighbors accuracy: 1.0000
SVM accuracy: 0.9000
DecisionTree accuracy: 0.9500
NaiveBayes accuracy: 0.9000
QDA accuracy: 0.9500

Selected top 3 classifiers: ['DecisionTree', 'QDA', 'KNeighbors']
0.0


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [43]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier


def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [44]:
def set_new_weights(model):
    predicted = model.predict(test_set)
    
    def calculate_accuracy_vector(predicted, labels):
        result = []
        for i in range(len(predicted)):
            if predicted[i] != labels[i]:  
                result.append(1)
            else:
                result.append(0)
        return np.array(result)
    
    I = calculate_accuracy_vector(predicted, test_labels)
    
    new_weights = 1 + I
    return new_weights / np.sum(new_weights)

Train the classifier with the code below:

In [45]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00132802 0.00066401 0.00132802 0.00132802 0.00066401 0.00132802
 0.00066401 0.00132802 0.00066401 0.00132802 0.00132802 0.00066401
 0.00066401 0.00066401 0.00132802 0.00066401 0.00066401 0.00132802
 0.00132802 0.00066401 0.00132802 0.00132802 0.00132802 0.00132802
 0.00066401 0.00066401 0.00066401 0.00132802 0.00132802 0.00066401
 0.00132802 0.00132802 0.00066401 0.00132802 0.00066401 0.00132802
 0.00066401 0.00132802 0.00132802 0.00066401 0.00066401 0.00132802
 0.00132802 0.00132802 0.00066401 0.00132802 0.00132802 0.00066401
 0.00132802 0.00132802 0.00066401 0.00132802 0.00066401 0.00066401
 0.00132802 0.00066401 0.00132802 0.00132802 0.00132802 0.00132802
 0.00132802 0.00066401 0.00066401 0.00066401 0.00066401 0.00066401
 0.00132802 0.00132802 0.00132802 0.00066401 0.00066401 0.00066401
 0.00132802 0.00132802 0.00132802 0.00132802 0.00132802 0.00132802
 0.00132802 0.00066401 0.00066401 0.00066401 0.00132802 0.00066401
 0.00066401 0.00066401 0.00132802 0.00066401 0.00132802 0.0013

Set the validation data set:

In [46]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [47]:
def get_prediction(x):
    votes = []
    for model in classifiers:
        prediction = model.predict(x)
        votes.append(prediction)
    
    votes = np.array(votes).flatten()
    
    unique_values, counts = np.unique(votes, return_counts=True)
    
    return unique_values[np.argmax(counts)]

Test it:

In [48]:
prediction = get_prediction(validate_x)

print(prediction)

0
